#Zadanie 7 (7 pkt)
Celem zadania jest zaimplementowanie dwóch wersji naiwnego klasyfikatora Bayesa.
* W pierwszej wersji należy dokonać dyskretyzacji danych - przedział wartości każdego atrybutu dzielimy na cztery równe przedziały i każdej ciągłej wartości atrybutu przypisujemy wartość dyskretną wynikająca z przynależności do danego przedziału.
* W drugiej wersji wartości likelihood wyliczamy z rozkładów normalnych o średnich i odchyleniach standardowych wynikających z wartości atrybutów.
Trening i test należy przeprowadzić dla zbioru Iris, tak jak w przypadku zadania z drzewem klasyfikacyjnym. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania klasyfikatorów dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Dyskretyzacja danych - **0.5 pkt**
* Implementacja funkcji rozkładu normalnego o zadanej średniej i odchyleniu standardowym. - **0.5 pkt**
* Implementacja naiwnego klasyfikatora Bayesa dla danych dyskretnych. - **2.0 pkt**
* Implementacja naiwnego klasyfikatora Bayesa dla danych ciągłych. - **2.5 pkt**
* Przeprowadzenie eksperymentów, wnioski i sposób ich prezentacji. - **1.5 pkt**

In [75]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
from collections import Counter
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

In [76]:
class NaiveBayes:
    def __init__(self):
        self.priors = {}
        self.likelihoods = {}

    def build_classifier(self, train_features, train_classes):
        #  a priori P(A)
        class_counter =Counter(train_classes)
        self.priors = dict(class_counter)
        total_samples = len(train_classes)
        for key in self.priors:
            self.priors[key] /= total_samples

        # prawdopodobieństwo P(B|A)
        p = np.zeros((len(set(train_classes)), train_features.shape[1], 4))
        for class_elem, feature in zip(train_classes, train_features):     
            j=0
            for value in feature:
                p[int(class_elem), j, int(value)] += 1
                j += 1

        # przygotowanie słownika
        self.likelihoods = {key: 1 for key in set(train_classes)}
        for key in self.likelihoods.keys():
            self.likelihoods[key] = {key2: 1 for key2 in range(len(train_features[0]))}
            for key2 in self.likelihoods[key].keys():
                self.likelihoods[key][key2] = {key3: 1 for key3 in range(len(set(train_features.flatten())))}

        
        for class_val in class_counter:
            for feature in train_features:       
                for j in range(len(feature)):
                    self.likelihoods[class_val][j][feature[j]] *= p[class_val, j, int(feature[j])] / class_counter[class_val]


    @staticmethod
    def data_discretization(data:np.ndarray):
        for i in range(data.shape[1]):
            bin_edges = np.linspace(data[:, i].min(), data[:, i].max(), 4)
            data[:, i] = np.digitize(data[:, i], bins=bin_edges, right=True)
        return data

    def predict(self, sample):
        posteriors = {}

        # posteriory P(A|B)
        for class_val, class_prob in self.priors.items():
            posterior = class_prob #class_prob -> a priori
            for i in range(len(sample)):
                posterior *= self.likelihoods[class_val][i][sample[i]]

            posteriors[class_val] = posterior
        return max(posteriors, key=posteriors.get)


class GaussianNaiveBayes:
    def __init__(self):
        self.priors = {}
        self.likelihoods = {}

    def build_classifier(self, train_features, train_classes):
        #  a priori P(A)
        self.priors = dict(Counter(train_classes))
        total_samples = len(train_classes)
        for key in self.priors:
            self.priors[key] /= total_samples

        # prawdopodobieństwo P(B|A)
        for class_val in set(train_classes):
            class_indices = np.where(train_classes == class_val)
            class_data = train_features[class_indices]

            means = np.mean(class_data, axis=0)
            stds = np.std(class_data, axis=0)

            self.likelihoods[class_val] = {'mean': means, 'std': stds}

    @staticmethod
    def normal_dist(x, mean, std):
        exponent = np.exp(-(np.power(x - mean, 2) / (2 * np.power(std, 2))))
        return (1 / (np.sqrt(2 * np.pi) * std)) * exponent

    def predict(self, sample):
        posteriors = {}

        for class_val, class_prob in self.priors.items():
            posterior = class_prob
            for i in range(len(sample)):
                posterior *= GaussianNaiveBayes.normal_dist(sample[i],
                                                              self.likelihoods[class_val]['mean'][i],
                                                              self.likelihoods[class_val]['std'][i])
            posteriors[class_val] = posterior
        return max(posteriors, key=posteriors.get)


In [77]:
iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=153)

nb = NaiveBayes()

x_train_discrete = NaiveBayes.data_discretization(x_train)

nb.build_classifier(x_train_discrete, y_train)

x_test_discrete = NaiveBayes.data_discretization(x_test)

predictions = [nb.predict(sample) for sample in x_test_discrete]

print(f'Accuracy: {accuracy_score(y_test, predictions)}')
print('Classification Report:')
print(classification_report(y_test, predictions))


Accuracy: 0.9666666666666667
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       0.88      1.00      0.93         7
           2       1.00      0.92      0.96        12

    accuracy                           0.97        30
   macro avg       0.96      0.97      0.96        30
weighted avg       0.97      0.97      0.97        30



In [78]:
iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

nb = GaussianNaiveBayes()

nb.build_classifier(x_train, y_train)

predictions = [nb.predict(sample) for sample in x_test]

print(f'Accuracy: {accuracy_score(y_test, predictions)}')
print('Classification Report:')
print(classification_report(y_test, predictions))


Accuracy: 0.9666666666666667
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.86      1.00      0.92         6
           2       1.00      0.91      0.95        11

    accuracy                           0.97        30
   macro avg       0.95      0.97      0.96        30
weighted avg       0.97      0.97      0.97        30

